In [3]:
import rosbag
PATH="/home/xuhao/Dropbox/data/TRO2020-SwarmLocal/tro2020-bags/pre/"
PATHOUT="/home/xuhao/bags/swarm_local_2020_12/"
input_bag = PATH +"drone1-swarm_local_2020-12-13-21-31-51.bag"
output_bag = PATHOUT + "drone1-swarm_local_2020-12-13-21-31-51_processed.bag"
moveout_topics = {
    "/swarm_drones/node_detected",
    "/swarm_drones/swarm_drone_fused",
    "/swarm_drones/swarm_drone_fused_relative",
    "/swarm_drones/swarm_frame",
    "/swarm_drones/swarm_frame_predict"
}
#Moveout node_detected
with rosbag.Bag(output_bag, 'w') as outbag:
    for topic, msg, t in rosbag.Bag(input_bag).read_messages():
        if topic in moveout_topics:
            pass
        else:
            #outbag.write(topic, msg, msg.header.stamp if msg._has_header else t)
            outbag.write(topic, msg, t)            

In [1]:
import rosbag
import os
PATH="/media/xuhao/DataStorage/tro2020-data/"
PATHOUT="/home/xuhao/bags/swarm_local_2020_12/"
bags = ["drone1-swarm_local_raw_2020-12-12.bag",
                "drone2-swarm_local_raw_2020-12-12.bag"]

t0 = None
for bag in bags:
    for topic, msg, t in rosbag.Bag(PATH+bag).read_messages():
        print(f"Bag {bag} start at {t.to_sec()}")
        t0 = t
        break
print("Will use", t0.to_sec(), "as start")

ros_t0 = None
lps_t1 = None

# for bag in bags:
#     #cmd = f"/opt/ros/melodic/bin/rosbag filter {PATH}{bag} {PATHOUT}filtered-{bag} 't.to_sec() >= {t.to_sec()}'"
#     #print(os.popen(cmd).read())
#         for topic, msg, t in rosbag.Bag(PATH+bag).read_messages():
#             outbag.write(topic, msg, msg.header.stamp if msg._has_header else t)

ros_t0 = None
lps_t1 = None
for bag in bags:
    output_bag = PATHOUT+"filtered-"+bag;
    with rosbag.Bag(output_bag, 'w') as outbag:
        for topic, msg, t in rosbag.Bag(PATH+bag).read_messages():
            if t < t0:
                continue
            if bag ==  "drone2-swarm_local_raw_2020-12-12.bag":
                if topic == "/uwb_node/remote_nodes":
                    if ros_t0 is None:
                        ros_t0 = msg.header.stamp.to_sec()
                    if lps_t1 is None:
                        lps_t1 = msg.sys_time
                    lps_time = (msg.header.stamp.to_sec() - ros_t0)*1000 + lps_t1
                    #print(msg.sys_time, lps_time)
                    msg.sys_time = int(lps_time)
            outbag.write(topic, msg, msg.header.stamp if msg._has_header else t)


Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


Bag drone1-swarm_local_raw_2020-12-12.bag start at 1607768180.4341998
Bag drone2-swarm_local_raw_2020-12-12.bag start at 1607768182.173061
Will use 1607768182.173061 as start
